In [2]:
from collections import Counter
import pandas as pd

In [19]:
def tab2df(filepath):
    """convert BLAST tab output to pd.Dataframe."""

    #columns_lst=["qId", "tId", "seqIdentity", "alnLen", "mismatchCnt", "gapOpenCnt",
    #             "qStart", "qEnd", "tStart", "tEnd", "eVal", "bitScore"]
    #result_df=pd.read_csv(filepath, delimiter='\t', names=columns_lst)
    #result_df["category"]=[qId.split('|')[0] for qId in result_df["qId"]]
    
    columns_lst=["query_name","sbject_name","seqIdentity","alnLen","misMatchCnt","gapOpenCnt",
                 "query_start", "query_end", "sbjct_start","sbjct_end","evalue","bitScore"]
    result_df=pd.read_csv(filepath, delimiter='\t', names=columns_lst)
    result_df["category"]=[qId.split('|')[0][1:] for qId in result_df["query_name"]]#mmseqs has > in its name
    
    return result_df

In [18]:
def totalize_result(result_df):
    """totalize #hit for each category"""
    thres_lst=[1,0.1,0.01,1e-3,1e-4,1e-5]
    dct_lst=[]
    for thres in thres_lst:
        dct={}
        dct["thres"]=thres
        filtered_df=result_df[result_df["evalue"]<thres]
        #filtered_df=filtered_df.drop_duplicates(subset=['qseqid'])
        dct.update(Counter(filtered_df["category"]))
        dct_lst.append(dct)
    count_df=pd.DataFrame(dct_lst)
    
    count_df=count_df.fillna(0)
    name_lst=[]
    for prefix in ["F","DSF","DTSF"]:
        for i in range(2,7):
            name=prefix+str(i)
            name_lst.append(name)
            
    count_df[name_lst]=count_df[name_lst].astype(int)
    count_df=count_df[["thres"]+name_lst]
    
    return count_df

In [4]:
txtFilepath="../../createdatabase/out/genus_872.txt"
catalog_df=pd.read_csv(txtFilepath)
catalog_df.head()

,taxid,kingdom,phylum,class,order,family,genus,species,count_real,count_sim,diff,ftp_basename,organism_name,genetic_code,G+C
0,525146,-1,1224,28221,213115,194924,872,876,0.851789,0.777740,0.074048,GCF_000022125.1_ASM2212v1,Desulfovibrio desulfuricans subsp. desulfurica...,11,0.580722
1,883,-1,1224,28221,213115,194924,872,881,1.619893,1.407806,0.212088,GCF_000021385.1_ASM2138v1,Desulfovibrio vulgaris str. 'Miyazaki F',11,0.671090
2,901,-1,1224,28221,213115,194924,872,901,1.387866,1.183206,0.204661,GCF_900116045.1_DESPIGER,Desulfovibrio piger,11,0.641799
3,526222,-1,1224,28221,213115,194924,872,880,0.594646,0.536652,0.057994,GCF_000023445.1_ASM2344v1,Desulfovibrio salexigens DSM 2638,11,0.470928
4,641491,-1,1224,28221,213115,194924,872,876,1.622397,1.369687,0.252709,GCF_000189295.2_ASM18929v2,Desulfovibrio desulfuricans ND132,11,0.652094


In [20]:
for basename in catalog_df["ftp_basename"]:
    print()
    resultFilepath="/data/mitsuki/data/mmseqs/result/{}.m8".format(basename)
    print()
    print("PROCESSING {}".format(resultFilepath))

    result_df=tab2df(resultFilepath)
    count_df=totalize_result(result_df)
    print(count_df)



PROCESSING /data/mitsuki/data/mmseqs/result/GCF_000022125.1_ASM2212v1.m8
     thres   F2  F3   F4   F5  F6  DSF2  DSF3  DSF4  DSF5  DSF6  DTSF2  DTSF3  \
0  1.00000  113  92  104  103  19    81    60   115    83    22     85     68   
1  0.10000   34  29   15   21   7     9     5     9    12     1      9      7   
2  0.01000   19  15    5    6   2     0     2     0     2     0      0      2   
3  0.00100   15  12    4    3   2     0     0     0     0     0      0      0   
4  0.00010   10  11    4    3   2     0     0     0     0     0      0      0   
5  0.00001    7   8    4    2   2     0     0     0     0     0      0      0   

   DTSF4  DTSF5  DTSF6  
0    130    104     21  
1     13     13      2  
2      0      2      0  
3      0      1      0  
4      0      0      0  
5      0      0      0  


PROCESSING /data/mitsuki/data/mmseqs/result/GCF_000021385.1_ASM2138v1.m8
     thres   F2   F3   F4  F5  F6  DSF2  DSF3  DSF4  DSF5  DSF6  DTSF2  DTSF3  \
0  1.00000  140  148  134 

In [13]:
result_df.columns

Index(['query_name', 'sbject_name', 'seqIdentity', 'alnLen', 'misMatchCnt',
       'gapOpenCnt', 'query_start', 'query_end', 'sbjct_start', 'sbjct_end',
       'evalue', 'bitScore', 'category'],
      dtype='object')

In [15]:
result_df.head()

,query_name,sbject_name,seqIdentity,alnLen,misMatchCnt,gapOpenCnt,query_start,query_end,sbjct_start,sbjct_end,evalue,bitScore,category
0,>F3|lcl|NC_011883.1_cds_WP_041723838.1_456,WP_012624093.1,1.0,11.0,0.0,0.0,1.0,12.0,326.0,337.0,0.0398,31.0,>F3
1,AGQCGGPAPCPE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AGQCGGPAPCPE
2,AGQCGGPAPCPE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AGQCGGPAPCPE
3,>F3|lcl|NC_011883.1_cds_WP_041723838.1_456,WP_072336458.1,1.0,10.0,0.0,0.0,1.0,11.0,315.0,325.0,0.1980,29.0,>F3
4,AGQCGGPAPCP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AGQCGGPAPCP
